In [ ]:
import numpy as np
import gpflow
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
import matplotlib.pyplot as plt
import tensorflow as tf
from gpflow.utilities import print_summary, set_trainable, to_default_float, positive
import statistics
from scipy import stats
from scipy import io
from scipy.stats import gamma
import random
# from gpflow.utilities import print_summary, positive
import math
from gpflow.ci_utils import ci_niter
#import plot_from_samples, colors
colors = ["#1f77b4", "#ff7f0e", "#2ca02c"]

In [ ]:
# load data
annots = io.loadmat('ACU01dup')
annots2 = io.loadmat('ACU02dup')
annots3 = io.loadmat('ACU03dup')
annots4 = io.loadmat('ACU04dup')
annots5 = io.loadmat('ACU05dup')
annots6 = io.loadmat('ACU06dup')
annots7 = io.loadmat('ACU07dup')
annot = io.loadmat('LTV01dup')
annot2 = io.loadmat('LTV02dup')
annot3 = io.loadmat('LTV03dup')
annot4 = io.loadmat('LTV04dup')
annot5 = io.loadmat('LTV05dup')
annot6 = io.loadmat('LTV06dup')

In [ ]:
def ACU_data(i):
    
    if i == 1:
        t=np.squeeze(annots['ACU01dup'][0][0]['t'])
        genes=np.squeeze(annots['ACU01dup'][0][0]['genes'])
        rawdat=np.squeeze(annots['ACU01dup'][0][0]['rawdat'])
        dat=np.squeeze(annots['ACU01dup'][0][0]['dat'])
        
    elif i == 2:
        t=np.squeeze(annots2['ACU02dup'][0][0]['t'])
        genes=np.squeeze(annots2['ACU02dup'][0][0]['genes'])
        rawdat=np.squeeze(annots2['ACU02dup'][0][0]['rawdat'])
        dat=np.squeeze(annots2['ACU02dup'][0][0]['dat'])
        
    elif i == 3:
        t=np.squeeze(annots3['ACU03dup'][0][0]['t'])
        genes=np.squeeze(annots3['ACU03dup'][0][0]['genes'])
        rawdat=np.squeeze(annots3['ACU03dup'][0][0]['rawdat'])
        dat=np.squeeze(annots3['ACU03dup'][0][0]['dat'])
        
    elif i == 4:
        t=np.squeeze(annots4['ACU04dup'][0][0]['t'])
        genes=np.squeeze(annots4['ACU04dup'][0][0]['genes'])
        rawdat=np.squeeze(annots4['ACU04dup'][0][0]['rawdat'])
        dat=np.squeeze(annots4['ACU04dup'][0][0]['dat'])
        
    elif i == 5:
        t=np.squeeze(annots5['ACU05dup'][0][0]['t'])
        genes=np.squeeze(annots5['ACU05dup'][0][0]['genes'])
        rawdat=np.squeeze(annots5['ACU05dup'][0][0]['rawdat'])
        dat=np.squeeze(annots5['ACU05dup'][0][0]['dat'])
        
    elif i == 6:
        t=np.squeeze(annots6['ACU06dup'][0][0]['t'])
        genes=np.squeeze(annots6['ACU06dup'][0][0]['genes'])
        rawdat=np.squeeze(annots6['ACU06dup'][0][0]['rawdat'])
        dat=np.squeeze(annots6['ACU06dup'][0][0]['dat'])
        
    elif i == 7:
        t=np.squeeze(annots7['ACU07dup'][0][0]['t'])
        genes=np.squeeze(annots7['ACU07dup'][0][0]['genes'])
        rawdat=np.squeeze(annots7['ACU07dup'][0][0]['rawdat'])
        dat=np.squeeze(annots7['ACU07dup'][0][0]['dat'])
        
    return t, genes, rawdat, dat

In [ ]:
def LTV_data(i):
    
    if i == 1:
        t=np.squeeze(annot['LTV01dup'][0][0]['t'])
        genes=np.squeeze(annot['LTV01dup'][0][0]['genes'])
        rawdat=np.squeeze(annot['LTV01dup'][0][0]['rawdat'])
        dat=np.squeeze(annot['LTV01dup'][0][0]['dat'])
        
    elif i == 2:
        t=np.squeeze(annot2['LTV02dup'][0][0]['t'])
        genes=np.squeeze(annot2['LTV02dup'][0][0]['genes'])
        rawdat=np.squeeze(annot2['LTV02dup'][0][0]['rawdat'])
        dat=np.squeeze(annot2['LTV02dup'][0][0]['dat'])
        
    elif i == 3:
        t=np.squeeze(annot3['LTV03dup'][0][0]['t'])
        genes=np.squeeze(annot3['LTV03dup'][0][0]['genes'])
        rawdat=np.squeeze(annot3['LTV03dup'][0][0]['rawdat'])
        dat=np.squeeze(annot3['LTV03dup'][0][0]['dat'])
        
    elif i == 4:
        t=np.squeeze(annot4['LTV04dup'][0][0]['t'])
        genes=np.squeeze(annot4['LTV04dup'][0][0]['genes'])
        rawdat=np.squeeze(annot4['LTV04dup'][0][0]['rawdat'])
        dat=np.squeeze(annot4['LTV04dup'][0][0]['dat'])
        
    elif i == 5:
        t=np.squeeze(annot5['LTV05dup'][0][0]['t'])
        genes=np.squeeze(annot5['LTV05dup'][0][0]['genes'])
        rawdat=np.squeeze(annot5['LTV05dup'][0][0]['rawdat'])
        dat=np.squeeze(annot5['LTV05dup'][0][0]['dat'])
        
    elif i == 6:
        t=np.squeeze(annot6['LTV06dup'][0][0]['t'])
        genes=np.squeeze(annot6['LTV06dup'][0][0]['genes'])
        rawdat=np.squeeze(annot6['LTV06dup'][0][0]['rawdat'])
        dat=np.squeeze(annot6['LTV06dup'][0][0]['dat'])
        
    return t, genes, rawdat, dat

In [ ]:
def mean2(data):
    n=len(data)
    mean=sum(data)/n
    return mean

In [ ]:
class OffsetLinear(gpflow.kernels.Kernel):
    # define a linear kernel with  an offset
    def __init__(self, variance=1.0, offset=0.001, active_dims=None):
        super().__init__(active_dims)
        self.variance = gpflow.Parameter(variance, transform=positive())
        self.offset = gpflow.Parameter(offset, transform=positive())
        self._validate_ard_active_dims(self.variance)
        
    @property
    def ard(self) -> bool:
        """
        Whether ARD behaviour is active.
        """
        return self.variance.shape.ndims > 0

    def K(self, X, X2=None):
        if X2 is None:
            return tf.matmul(tf.subtract(X, self.offset) * self.variance, tf.subtract(X,self.offset), transpose_b=True)
        else:
            return tf.tensordot(tf.subtract(X, self.offset) * self.variance, tf.subtract(X2,self.offset), [[-1], [-1]])

    def K_diag(self, X):
        return tf.reduce_sum(tf.square(tf.subtract(X, self.offset)) * self.variance, axis=-1)

## Samples from OffsetLinear Kernel's Priors

In [ ]:
k_offset = OffsetLinear()
print_summary(k_offset, fmt="notebook")

In [ ]:
np.random.seed(30)  # for reproducibility

def plotkernelsample(k, ax, xmin=0, xmax=3):
    # function to plot prior samples from gpflow docs
    xx = np.linspace(xmin, xmax, 300)[:, None]
    K = k(xx)
    ax.plot(xx, np.random.multivariate_normal(np.zeros(300), K, 5).T)
    ax.set_title("Samples " + k.__class__.__name__)

In [ ]:
f, ax = plt.subplots(figsize=(12,6))
plotkernelsample(k_offset, ax)

In [ ]:
def shape_data(data, i):
    # convert data t a usable shape
    t, genes, rawdat, dat = data # extract data
    
    # concatenate duplicates
    rawdata = np.concatenate([rawdat[:,:,i][0],rawdat[:,:,i][1]])
    time = np.concatenate([t,t])
    
    # extract nans
    notnan = ~np.isnan(rawdata) # mark any numbers
    rawdata = rawdata[notnan] # extract numbers
    time = time[notnan] # extract corresponding times
    n = np.size(rawdata)
    
    # convert data to right shape
    X = time.reshape(n,1)
    X = tf.cast(X, tf.double)
    X.numpy()
    Z = rawdata.reshape(n,1)
    Z = tf.cast(Z, tf.double)
    Z.numpy()
    Z = stats.zscore(Z) # z-score raw data
    
    return X, Z

In [ ]:
def our_kernel(period=24.0, off_var=0.001, offset=0.001):
    
    # define kernel
    k1=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=period) # periodic kernel
    k2=OffsetLinear(variance=off_var, offset=offset) # offset kernel
    k3=gpflow.kernels.Constant(0.001)
    k=k1+k2+k3
    
    return k

## Samples from Combined Kernel's Priors

In [ ]:
np.random.seed(23)
k = our_kernel()
f, ax = plt.subplots(figsize=(12,6))
plotkernelsample(k, ax, xmax = 50)
plt.title("Samples from Priors of Combined Kernel")

In [ ]:
def plot_mod(X, Z, m, pl_start = -5, pl_end = 55):
    
    ## generate test points for prediction
    xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)

    ## predict mean and variance of latent GP at test points
    mean, var = m.predict_f(xx) # predict mean and variance from model

    ## generate 10 samples from posterior
    tf.random.set_seed(1)  # for reproducibility
    samples = m.predict_f_samples(xx, 10)  # predict samples

    ## plot
    plt.figure(figsize=(12, 6))
    plt.plot(X, Z, "kx", mew=2) # plot data
    plt.plot(xx, mean, "C0", lw=2) # plot predicted mean
    plt.fill_between(
        xx[:, 0],
        mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
        mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
        color="C0",
        alpha=0.2,
    ) # plot uncertainty window

    plt.plot(xx, samples[:, :, 0].numpy().T, "C0", linewidth=0.5)
    _ = plt.xlim(pl_start, pl_end)
    plt.title("Samples from Posterior Distribution")
    plt.show()

In [ ]:
def load_priors(k):
    
    k.kernels[0].base_kernel.variance.prior = tfp.distributions.Gamma(
        to_default_float(2), to_default_float(1)
    ) 

    k.kernels[0].base_kernel.lengthscales.prior = tfp.distributions.Gamma(
        to_default_float(2), to_default_float(1)
    )
    k.kernels[0].period.prior = tfp.distributions.Gamma(
       to_default_float(25), to_default_float(1)
    )
    k.kernels[1].variance.prior = tfp.distributions.Gamma(
       to_default_float(2), to_default_float(1000)
    )
    k.kernels[1].offset.prior = tfp.distributions.Gamma(
       to_default_float(2), to_default_float(1000)
    )
    k.kernels[2].variance.prior = tfp.distributions.Gamma(
       to_default_float(2), to_default_float(1000)
    )
    return k

In [ ]:
def mle_sub(data, i, period=24.0, off_var=0.001, offset=0.001, plot_m=False):
    
    # function to compute MLE for ACU patient with time subtracted mean
    X, Z = shape_data(data, i)
    
    # subtract mean
    X_orig = X
    X = X_orig - mean2(X_orig)
    
    k = our_kernel(period, off_var, offset)
    
    # create model using data and kernel
    m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
    m.likelihood.variance.assign(0.1)
    
    # optimise using MLE
    opt = gpflow.optimizers.Scipy()
    opt.minimize(m.training_loss, variables=m.trainable_variables)
    
    if plot_m:
        # return hyperparameters
        print("Maximum Likelihood Estimate for gene", i+1)
        print_summary(m, fmt="notebook")
        
        # plot
        plot_mod(X, Z, m, pl_start = -25, pl_end = 25)
    return X, Z, m

## Using priors with Gamma distribution

A random quantity $X$ has a Gamma distribution with parameters $\alpha \geq 1$ and $\beta$, written $X | \alpha, \beta ~$ Gamma($\alpha, \beta$), if it has pdf
\begin{equation}
f(x | \alpha,\beta) = 
\begin{cases} 
  \frac{\beta^{\alpha}}{\Gamma(\alpha)}x^{\alpha - 1}\exp({-\beta x}) & x\in [0,\infty]\\
  0 & otherwise \\
\end{cases}
\end{equation}
This is useful when modelling strictly positive quantities, as we want our prior to be positive, we will try using a Gamma prior. To choose the right prior values, we can see what each distribution looks like below.

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(14, 8), constrained_layout=True)
x = np.linspace(0,10,100)
axs[0, 0].plot(x, gamma.pdf(x, 2, scale=1),label='$\Gamma(2,1)$')
axs[0, 0].legend(loc='best', frameon=False)
axs[0, 0].set_title("Gamma Prior for Signal Variance and Lengthscale: $\Gamma(2,1)$")
x = np.linspace(0,48,100)
axs[0, 1].plot(x, gamma.pdf(x, 25, scale=1),label='$\Gamma(25,1)$')
axs[0, 1].legend(loc='best', frameon=False)
axs[0, 1].set_title("Gamma Prior for Period: $\Gamma(25,1)$")
x = np.linspace(0,0.5,100)
axs[1, 0].plot(x, gamma.pdf(x, 2, scale=0.1),label='$\Gamma(2,0.1)$')
axs[1, 0].legend(loc='best', frameon=False)
axs[1, 0].set_title("Gamma Prior for Likelihood Variance: $\Gamma(2,0.1)$")
x = np.linspace(0,0.1,100)
axs[1, 1].plot(x, gamma.pdf(x, 2, scale=0.001),label='$\Gamma(2,0.001)$')
axs[1, 1].legend(loc='best', frameon=False)
axs[1, 1].set_title("Gamma Prior for the Offset and Constant kernels: $\Gamma(2,0.001)$")
plt.show()

In [ ]:
def map_sub(data, i, period=24.0, off_var=0.001, offset=0.001, plot_m=False):
    # function for MAP of ACU patient with time subtracted mean
    
    X, Z = shape_data(data, i)
    
    # subtract mean time from time
    X_orig = X
    X = X_orig - mean2(X_orig)
    
    k = our_kernel(period, off_var, offset)
    
    # set kernel priors
    k = load_priors(k)
    
    # create model using data and kernel
    m = gpflow.models.GPR(data=(X,Z), kernel=k, mean_function=None)
    
    # set model priors
    m.likelihood.variance.prior = tfp.distributions.Gamma(
       to_default_float(2), to_default_float(10)
    )
        
    # optimise using MLE
    opt = gpflow.optimizers.Scipy()
    opt.minimize(m.training_loss, variables=m.trainable_variables)
    
    if plot_m:
        # return hyperparameters
        print("Maximum A Posteriori Estimate for gene", i+1)
        print_summary(m, fmt="notebook")
        
        # plot
        plot_mod(X, Z, m, pl_start = -25, pl_end = 25)
    return X, Z, m

# ACU01 Patient

In [ ]:
for i in range(13):
    print("Gene ", i+1)
    # plot data
    t, genes, rawdat, dat = ACU_data(1)
    time = np.concatenate([t,t])
    rawdata = np.concatenate([rawdat[:,:,i][0],rawdat[:,:,i][1]])
    plt.plot(t,rawdat[:,:,i][0])
    plt.plot(t,rawdat[:,:,i][1])
    plt.title("Raw Data")
    plt.show()
        
    # mle estimation
    mle_sub(ACU_data(1), i, plot_m=True)
    
    # map estimation
    map_sub(ACU_data(1), i, plot_m=True)

# ACU02 Patient

In [ ]:
for i in range(13):
    print("Gene ", i+1)
    # plot data
    t, genes, rawdat, dat = ACU_data(2)
    time = np.concatenate([t,t])
    rawdata = np.concatenate([rawdat[:,:,i][0],rawdat[:,:,i][1]])
    plt.plot(t,rawdat[:,:,i][0])
    plt.plot(t,rawdat[:,:,i][1])
    plt.title("Raw Data")
    plt.show()
    
    # mle estimation
    #mle_sub(ACU_data(2), i, plot_m=True)
    
    # map estimation
    map_sub(ACU_data(2), i, plot_m=True)

# ACU03 Patient

In [ ]:
for i in range(13):
    print("Gene ", i+1)
    # plot data
    t, genes, rawdat, dat = ACU_data(3)
    time = np.concatenate([t,t])
    rawdata = np.concatenate([rawdat[:,:,i][0],rawdat[:,:,i][1]])
    plt.plot(t,rawdat[:,:,i][0])
    plt.plot(t,rawdat[:,:,i][1])
    plt.title("Raw Data")
    plt.show()
    
    # mle estimation
    #mle_sub(ACU_data(3), i, plot_m=True)
    
    # map estimation
    map_sub(ACU_data(3), i, plot_m=True)

# ACU04 Patient

In [ ]:
for i in range(13):
    print("Gene ", i+1)
    # plot data
    t, genes, rawdat, dat = ACU_data(4)
    time = np.concatenate([t,t])
    rawdata = np.concatenate([rawdat[:,:,i][0],rawdat[:,:,i][1]])
    plt.plot(t,rawdat[:,:,i][0])
    plt.plot(t,rawdat[:,:,i][1])
    plt.title("Raw Data")
    plt.show()
    
    # mle estimation
    #mle_sub(ACU_data(4), i, plot_m=True)
    
    # map estimation
    map_sub(ACU_data(4), i, plot_m=True)

# ACU05 Patient

In [ ]:
for i in range(13):
    print("Gene ", i+1)
    # plot data
    t, genes, rawdat, dat = ACU_data(5)
    time = np.concatenate([t,t])
    rawdata = np.concatenate([rawdat[:,:,i][0],rawdat[:,:,i][1]])
    plt.plot(t,rawdat[:,:,i][0])
    plt.plot(t,rawdat[:,:,i][1])
    plt.title("Raw Data")
    plt.show()
    
    # mle estimation
    #mle_sub(ACU_data(5), i, plot_m=True)
    
    # map estimation
    map_sub(ACU_data(5), i, plot_m=True)

# ACU06 Patient

In [ ]:
for i in range(13):
    print("Gene ", i+1)
    # plot data
    t, genes, rawdat, dat = ACU_data(6)
    time = np.concatenate([t,t])
    rawdata = np.concatenate([rawdat[:,:,i][0],rawdat[:,:,i][1]])
    plt.plot(t,rawdat[:,:,i][0])
    plt.plot(t,rawdat[:,:,i][1])
    plt.title("Raw Data")
    plt.show()
    
    # mle estimation
    #mle_sub(ACU_data(6), i, plot_m=True)
    
    # map estimation
    map_sub(ACU_data(6), i, plot_m=True)

# ACU07 Patient

In [ ]:
for i in range(13):
    print("Gene ", i+1)
    # plot data
    t, genes, rawdat, dat = ACU_data(7)
    time = np.concatenate([t,t])
    rawdata = np.concatenate([rawdat[:,:,i][0],rawdat[:,:,i][1]])
    plt.plot(t,rawdat[:,:,i][0])
    plt.plot(t,rawdat[:,:,i][1])
    plt.title("Raw Data")
    plt.show()
    
    # mle estimation
    #mle_sub(ACU_data(7), i, plot_m=True)
    
    # map estimation
    map_sub(ACU_data(7), i, plot_m=True)

# LTV01 Patient

In [ ]:
for i in range(13):
    print("Gene ", i+1)
    # plot data
    t, genes, rawdat, dat = LTV_data(1)
    time = np.concatenate([t,t])
    rawdata = np.concatenate([rawdat[:,:,i][0],rawdat[:,:,i][1]])
    plt.plot(t,rawdat[:,:,i][0])
    plt.plot(t,rawdat[:,:,i][1])
    plt.title("Raw Data")
    plt.show()
    
    # mle estimation
    #mle_sub(LTV_data(1), i, plot_m=True)
    
    # map estimation
    map_sub(LTV_data(1), i, plot_m=True)

# LTV02 Patient

In [ ]:
for i in range(13):
    print("Gene ", i+1)
    # plot data
    t, genes, rawdat, dat = LTV_data(2)
    time = np.concatenate([t,t])
    rawdata = np.concatenate([rawdat[:,:,i][0],rawdat[:,:,i][1]])
    plt.plot(t,rawdat[:,:,i][0])
    plt.plot(t,rawdat[:,:,i][1])
    plt.title("Raw Data")
    plt.show()
    
    # mle estimation
    #mle_sub(LTV_data(2), i, plot_m=True)
    
    # map estimation
    map_sub(LTV_data(2), i, plot_m=True)

# LTV03 Patient

In [ ]:
for i in range(13):
    print("Gene ", i+1)
    # plot data
    t, genes, rawdat, dat = LTV_data(3)
    time = np.concatenate([t,t])
    rawdata = np.concatenate([rawdat[:,:,i][0],rawdat[:,:,i][1]])
    plt.plot(t,rawdat[:,:,i][0])
    plt.plot(t,rawdat[:,:,i][1])
    plt.title("Raw Data")
    plt.show()
    
    # mle estimation
    #mle_sub(LTV_data(3), i, plot_m=True)
    
    # map estimation
    map_sub(LTV_data(3), i, plot_m=True)

# LTV04 Patient

In [ ]:
for i in range(13):
    print("Gene ", i+1)
    # plot data
    t, genes, rawdat, dat = LTV_data(4)
    time = np.concatenate([t,t])
    rawdata = np.concatenate([rawdat[:,:,i][0],rawdat[:,:,i][1]])
    plt.plot(t,rawdat[:,:,i][0])
    plt.plot(t,rawdat[:,:,i][1])
    plt.title("Raw Data")
    plt.show()
    
    # mle estimation
    #mle_sub(LTV_data(4), i, plot_m=True)
    
    # map estimation
    map_sub(LTV_data(4), i, plot_m=True)

# LTV05 Patient

In [ ]:
for i in range(13):
    print("Gene ", i+1)
    # plot data
    t, genes, rawdat, dat = LTV_data(5)
    time = np.concatenate([t,t])
    rawdata = np.concatenate([rawdat[:,:,i][0],rawdat[:,:,i][1]])
    plt.plot(t,rawdat[:,:,i][0])
    plt.plot(t,rawdat[:,:,i][1])
    plt.title("Raw Data")
    plt.show()
    
    # mle estimation
    #mle_sub(LTV_data(5), i, plot_m=True)
    
    # map estimation
    map_sub(LTV_data(5), i, plot_m=True)

# LTV06 Patient

In [ ]:
for i in range(13):
    print("Gene ", i+1)
    # plot data
    t, genes, rawdat, dat = LTV_data(6)
    time = np.concatenate([t,t])
    rawdata = np.concatenate([rawdat[:,:,i][0],rawdat[:,:,i][1]])
    plt.plot(t,rawdat[:,:,i][0])
    plt.plot(t,rawdat[:,:,i][1])
    plt.title("Raw Data")
    plt.show()
    
    # mle estimation
    #mle_sub(LTV_data(6), i, plot_m=True)
    
    # map estimation
    map_sub(LTV_data(6), i, plot_m=True)

# Peak to Peak Amplitude

In [ ]:
def peak2peak_amp(data, i):
    
    X, Z, m = map_sub(data, i)
    
    # extract periodic kernel
    k1=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)
    m1 = gpflow.models.GPR(data=(X,Z), kernel=k1, mean_function=None)
    m1.kernel.base_kernel.variance.assign(m.kernel.kernels[0].base_kernel.variance)
    m1.kernel.base_kernel.lengthscales.assign(m.kernel.kernels[0].base_kernel.lengthscales)
    m1.kernel.period.assign(m.kernel.kernels[0].period)
    
    ## generate test points for prediction
    xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

    ## predict mean and variance of latent GP at test points
    mean1, var1 = m1.predict_f(xx)

    ## generate 10 samples from posterior
    tf.random.set_seed(1)  # for reproducibility
    samples = m1.predict_f_samples(xx, 10)  # shape (10, 100, 1)

    ## plot
    plt.figure(figsize=(12, 6))
    plt.plot(xx, mean1, "C0", lw=2)

    _ = plt.xlim(-25, 25)
    plt.show()
    
    # calculate peak to peak amplitude
    maxi = tf.math.reduce_max(mean1)
    mini = tf.math.reduce_min(mean1)
    amp = maxi-mini
    
    var = m1.kernel.base_kernel.variance
    return amp, var

In [ ]:
def mod_corr(data):
    
    amps = np.zeros(13)
    std = np.zeros(13)
    for i in range(13):
        amps1, std1 = peak2peak_amp(data, i)
        amps[i] = amps1
        std[i] = math.sqrt(std1.numpy())
    plt.scatter(amps,std)
    plt.title("Correlation between Peak to Peak Amplitude and Signal Standard Deviation")
    plt.xlabel("Peak to Peak Amplitude")
    plt.ylabel("Signal Standard Deviation")
    
    return amps, std

### Compare phase of Combined Kernel with only periodic Kernel

In [ ]:
X, Z, m = map_sub(ACU_data(1), 0)
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)
mean, var = m.predict_f(xx)

k1=gpflow.kernels.Periodic(gpflow.kernels.SquaredExponential(), period=24.0)
m1 = gpflow.models.GPR(data=(X,Z), kernel=k1, mean_function=None)
m1.kernel.base_kernel.variance.assign(m.kernel.kernels[0].base_kernel.variance)
m1.kernel.base_kernel.lengthscales.assign(m.kernel.kernels[0].base_kernel.lengthscales)
m1.kernel.period.assign(m.kernel.kernels[0].period)
m1.likelihood.variance.assign(m.likelihood.variance)
## generate test points for prediction
xx = np.linspace(-72.1, 72.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean1, var1 = m1.predict_f(xx)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m1.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(xx, mean, "C0", lw=2)
plt.plot(xx, mean1, "C1", lw=2)

_ = plt.xlim(-25, 60)
plt.legend(["Combined Kernel", "Periodic Kernel Only"])
plt.title("Comparing phase of Extracted Periodic Kernel")
plt.show()

## ACU01

In [ ]:
mod_corr(ACU_data(1))

## ACU02

In [ ]:
mod_corr(ACU_data(2))

## ACU03

In [ ]:
mod_corr(ACU_data(3))

## ACU04

In [ ]:
mod_corr(ACU_data(4))

## ACU05

In [ ]:
mod_corr(ACU_data(5))

## ACU06

In [ ]:
mod_corr(ACU_data(6))

## ACU07

In [ ]:
mod_corr(ACU_data(7))

## LTV01

In [ ]:
mod_corr(LTV_data(1))

## LTV02

In [ ]:
mod_corr(LTV_data(2))

## LTV03

In [ ]:
mod_corr(LTV_data(3))

## LTV04

In [ ]:
mod_corr(LTV_data(4))

## LTV05

In [ ]:
mod_corr(LTV_data(5))

## LTV06

In [ ]:
mod_corr(LTV_data(6))

### Correlation of all ACU cohort

In [ ]:
# correlation of all ACU cohort
amps_a = np.zeros((7,13))
std_a = np.zeros((7,13))
for j in range(1,8):
    amps = np.zeros(13)
    std = np.zeros(13)
    for i in range(13):
        amps1, std1 = peak2peak_amp(ACU_data(j), i)
        amps[i] = amps1
        std[i] = math.sqrt(std1.numpy())
    amps_a[j-1]=amps
    std_a[j-1]=std
amps_a=amps_a.tolist()
std_a=std_a.tolist()

In [ ]:
# plot correlation
plt.figure(figsize=(10,6))
plt.scatter(amps_a,std_a)
plt.title("Correlation between Peak to Peak Amplitude and Signal Standard Deviation for ACU cohort")
plt.xlabel("Peak to Peak Amplitude")
plt.ylabel("Signal Standard Deviation")
plt.show()

### Correlation of all LTV cohort

In [ ]:
# correlation of all LTV cohort
amps_l = np.zeros((6,13))
std_l = np.zeros((6,13))
for j in range(1,7):
    amps = np.zeros(13)
    std = np.zeros(13)
    for i in range(13):
        amps1, std1 = peak2peak_amp(LTV_data(j), i)
        amps[i] = amps1
        std[i] = math.sqrt(std1.numpy())
    amps_l[j-1]=amps
    std_l[j-1]=std
amps_l=amps_l.tolist()
std_l=std_l.tolist()

In [ ]:
# plot correlation
plt.figure(figsize=(10,6))
plt.scatter(amps_l,std_l)
plt.title("Correlation between Peak to Peak Amplitude and Signal Standard Deviation for LTV cohort")
plt.xlabel("Peak to Peak Amplitude")
plt.ylabel("Signal Standard Deviation")
plt.show()

## Measures of signalling strength and circadian nature

In [ ]:
def Ca(data, i):
    
    # calculates measure of oscillation strength
    X, Z, m = map_sub(data, i)
    l_var = m.likelihood.variance.numpy()
    sig_var = m.kernel.kernels[0].base_kernel.variance.numpy()
    Ca = sig_var / (l_var + sig_var)
    
    return Ca

In [ ]:
def Cp(data, i):
    
    # measure of how circadian the period is
    X, Z, m = map_sub(data, i)
    #lscales = m.kernel.kernels[0].base_kernel.lengthscales.numpy()
    period = m.kernel.kernels[0].period
    #Cp = lscales/((period.numpy()-24)**2 + 1)
    Cp = 1/((period.numpy()-24)**2 + 1)
    
    return Cp

In [ ]:
def C_measure(data, i):
    
    # overall measure of gene
    C_measure = Ca(data,i)*Cp(data,i)
    
    return C_measure

In [ ]:
def C_plot_list(data):
    # function to create an array of measures and plot scatter plot of them
    Ca_list = np.zeros(13) # empty list for Ca measure
    Cp_list = np.zeros(13)
    C_measure_list = np.zeros(13)
    for i in range(13):
        Ca_list[i] = Ca(data, i)
        Cp_list[i] = Cp(data, i)
        C_measure_list[i] = C_measure(data, i)

    plt.scatter(Ca_list, Cp_list)
    plt.xlabel("C_a")
    plt.ylabel("C_p")

    return C_measure_list

## Scattergram of measures

### ACU patients on one plot

In [ ]:
C_pl_a = np.zeros(shape=(7,13)) # an array to store C_measure
plt.figure(figsize=(6,6))
plt.xlim([0, 1])
plt.ylim([0, 1])
for i in range(1, 8):
    C_pl_a[i-1]=C_plot_list(ACU_data(i))
plt.title("C_a vs C_p for ACU dupdat")
plt.show()
C_pl_a

### ACU patients on separate plots

In [ ]:
'''for i in range(1, 8):
    C_plot_list(ACU_data(i))
    print("ACU Patient", i)
    plt.show()'''

### LTV patients on one plot

In [ ]:
C_pl_l = np.zeros(shape=(6,13)) # an array to store C_measure
plt.figure(figsize=(6,6))
plt.xlim([0, 1])
plt.ylim([0, 1])
for i in range(1, 7):
    C_pl_l[i-1] = C_plot_list(LTV_data(i))
plt.title("C_a vs C_p for LTV dupdat")
plt.show()
C_pl_l

### LTV patients on separate plots

In [ ]:
'''for i in range(1, 7):
    C_plot_list(LTV_data(i))
    print("LTV Patient", i)
    plt.show()'''

## Graph for each gene, with different patient values

In [ ]:
'''for i in range(13):
    plt.scatter(C_pl_l[:,i])
    plt.scatter(C_pl_a[:,i])
    plt.show()'''

### Maximum Score for ACU patient

In [ ]:
np.max(C_pl_a)

In [ ]:
C_pl_a

## New attempt

In [ ]:
def C_measure_new(data, i):
    
    # overall measure of gene
    X, Z, m = map_sub(data, i)
    lscales = m.kernel.kernels[0].base_kernel.lengthscales.numpy()
    C_measure_new = 0.5*(Ca(data,i)+Cp(data,i))*lscales
    
    return C_measure_new

In [ ]:
def C_measure_list(data):
    # function to create an array of measures and plot scatter plot of them
    C_measure_new_list = np.zeros(13)
    for i in range(13):
        C_measure_new_list[i] = C_measure_new(data, i)
        
    return C_measure_new_list

## ACU patients

In [ ]:
C_pl_new_a = np.zeros(shape=(7,13)) # an array to store C_measure
for i in range(1, 8):
    C_pl_new_a[i-1]=C_measure_list(ACU_data(i))

### Best score in ACU

In [ ]:
np.max(C_pl_new_a)

In [ ]:
np.where(C_pl_new_a == np.max(C_pl_new_a))

In [ ]:
C_pl_new_a

In [ ]:
map_sub(ACU_data(2), 3, plot_m=True)

### Worst score in ACU

In [ ]:
np.min(C_pl_new_a)

In [ ]:
np.where(C_pl_new_a == np.min(C_pl_new_a))

In [ ]:
map_sub(ACU_data(4), 7, plot_m=True)

## LTV patients

In [ ]:
C_pl_new_l = np.zeros(shape=(6,13)) # an array to store C_measure
for i in range(1, 7):
    C_pl_new_l[i-1]=C_measure_list(LTV_data(i))

### Best score in LTV

In [ ]:
np.max(C_pl_new_l)

In [ ]:
np.where(C_pl_new_l == np.max(C_pl_new_l))

In [ ]:
C_pl_new_l

In [ ]:
map_sub(LTV_data(6), 8, plot_m=True)

### Worst score in LTV

In [ ]:
np.min(C_pl_new_l)

In [ ]:
np.where(C_pl_new_l == np.min(C_pl_new_l))

In [ ]:
map_sub(LTV_data(6), 0, plot_m=True)

## New attempt 2

In [ ]:
def C_measure_new2(data, i):
    
    # overall measure of gene
    X, Z, m = map_sub(data, i)
    lscales = m.kernel.kernels[0].base_kernel.lengthscales.numpy()
    if lscales>=0.5:
        C_measure_new2 = Ca(data,i)*Cp(data,i)
    else:
        C_measure_new2 = 0
    
    return C_measure_new2

In [ ]:
def C_measure_list2(data):
    # function to create an array of measures and plot scatter plot of them
    C_measure_new_list2 = np.zeros(13)
    for i in range(13):
        C_measure_new_list2[i] = C_measure_new2(data, i)
        
    return C_measure_new_list2

## ACU patients

In [ ]:
C_pl_new_a2 = np.zeros(shape=(7,13)) # an array to store C_measure
for i in range(1, 8):
    C_pl_new_a2[i-1]=C_measure_list2(ACU_data(i))

### Best score in ACU

In [ ]:
np.max(C_pl_new_a2)

In [ ]:
np.where(C_pl_new_a2 == np.max(C_pl_new_a2))

In [ ]:
C_pl_new_a2

In [ ]:
map_sub(ACU_data(3), 10, plot_m=True)

## LTV patients

In [ ]:
C_pl_new_l2 = np.zeros(shape=(6,13)) # an array to store C_measure
for i in range(1, 7):
    C_pl_new_l2[i-1]=C_measure_list2(LTV_data(i))

### Best score in LTV

In [ ]:
np.max(C_pl_new_l2)

In [ ]:
np.where(C_pl_new_l2 == np.max(C_pl_new_l2))

In [ ]:
C_pl_new_l2

In [ ]:
map_sub(LTV_data(4), 3, plot_m=True)

## New attempt 3

In [ ]:
def C_measure_new3(data, i):
    
    # overall measure of gene
    X, Z, m = map_sub(data, i)
    lscales = m.kernel.kernels[0].base_kernel.lengthscales.numpy()
    if lscales>=1:
        C_measure_new3 = Ca(data,i)*Cp(data,i)
    else:
        C_measure_new3 = Ca(data,i)*Cp(data,i)*lscales
    
    return C_measure_new3

In [ ]:
def C_measure_list3(data):
    # function to create an array of measures and plot scatter plot of them
    C_measure_new_list3 = np.zeros(13)
    for i in range(13):
        C_measure_new_list3[i] = C_measure_new3(data, i)
        
    return C_measure_new_list3

## ACU patients

In [ ]:
C_pl_new_a3 = np.zeros(shape=(7,13)) # an array to store C_measure
for i in range(1, 8):
    C_pl_new_a3[i-1]=C_measure_list3(ACU_data(i))

### Best score in ACU

In [ ]:
np.max(C_pl_new_a3)

In [ ]:
np.where(C_pl_new_a3 == np.max(C_pl_new_a3))

In [ ]:
map_sub(ACU_data(3), 10, plot_m=True)

In [ ]:
np.min(C_pl_new_a3)

## LTV patients

In [ ]:
C_pl_new_l3 = np.zeros(shape=(6,13)) # an array to store C_measure
for i in range(1, 7):
    C_pl_new_l3[i-1]=C_measure_list3(LTV_data(i))

### Best score in LTV

In [ ]:
np.max(C_pl_new_l3)

In [ ]:
C_pl_new_l3

In [ ]:
np.where(C_pl_new_l3 == np.max(C_pl_new_l3))

In [ ]:
map_sub(LTV_data(4), 3, plot_m=True)

In [ ]:
np.min(C_pl_new_l3)

In [ ]:
np.where(C_pl_new_l3 == np.min(C_pl_new_l3))

### Maximum Score for LTV patient

In [ ]:
np.max(C_pl_l)

In [ ]:
C_pl_l

## Box Plots for Period

### ACU patients

In [ ]:
periods_a = np.zeros((7,13)) # create an empty array for ACU periods
for j in range(1,8): # loop over patients
    period = np.zeros(13) # empty array for an individual's periods
    for i in range(13): # loop over genes
        X, Z, m = map_sub(ACU_data(j),i) # map estimation
        period[i]=m.kernel.kernels[0].period.numpy() # assign period
    periods_a[j-1]=period

### Finding outliers in the data

In [ ]:
x = np.where(periods_a > 30) # find periods over 30 hours

print(x)

In [ ]:
y = np.where(periods_a < 19) # find periods under 19 hours

print(y)

In [ ]:
periods_a

In [ ]:
# extracting labels for genes
genes=np.squeeze(annot['LTV01dup'][0][0]['genes'])

In [ ]:
# box plots of ACU patients
plt.figure(figsize=(12,8))
plt.boxplot(periods_a, labels=genes)
plt.axhline(y=24, linestyle='-') # show circadian period
plt.title("Spread of Periods among ACU patients (Duplicate)")
plt.xlabel("Genes")
plt.yticks(np.arange(14,36))
plt.ylabel("Period (hr)")
plt.ylim([14,36])

In [ ]:
# create array to assign periods to correct patient
a = np.arange(1,14)
ad_a = np.concatenate((a,a,a,a,a,a,a))
lists_a=np.reshape((periods_a),np.shape(ad_a)) # reshape for scatter

In [ ]:
# violin plots of ACU period spread
plt.figure(figsize=(12,8))
plt.violinplot(periods_a, showmeans=True) # show means on periods
plt.axhline(y=24, linestyle='-') # show circadian period
plt.scatter(ad_a,lists_a,c='k') # add individual data points
plt.title("Spread of Periods among ACU patients (Duplicate)")
plt.xlabel("Genes")
plt.xticks(np.arange(1,14), labels=genes)
plt.yticks(np.arange(14,37))
plt.ylabel("Period (hr)")
plt.ylim([14,36])

### LTV patients

In [ ]:
periods_l = np.zeros((6,13)) # create an erray to store LTV periods in
for j in range(1,7): #loop over patients
    period = np.zeros(13) # empty erray for individual's genes
    for i in range(13): # loop over genes
        X, Z, m = map_sub(LTV_data(j),i) # map estimatino
        period[i]=m.kernel.kernels[0].period.numpy() # assign period
    periods_l[j-1]=period

In [ ]:
# box plots of periods for LTV
plt.figure(figsize=(12,8))
plt.boxplot(periods_l, labels=genes)
plt.axhline(y=24, linestyle='-')
plt.title("Spread of Periods among LTV patients (Duplicate)")
plt.xlabel("Genes")
plt.yticks(np.arange(14,36))
plt.ylabel("Period (hr)")
plt.ylim([14,36])

In [ ]:
# create an array to label each period to allow a scatter plot
a = np.arange(1,14)
ad_l = np.concatenate((a,a,a,a,a,a))
lists_l=np.reshape((periods_l),np.shape(ad_l)) # reshape the periods for scatter plot

In [ ]:
# plot violin plot of periods in LTV cohort
plt.figure(figsize=(12,8))
plt.violinplot(periods_l, showmeans=True) # plot LTV periods and show means
plt.axhline(y=24, linestyle='-') # add line to show circadian period
plt.scatter(ad_l,lists_l,c='k') # plot individual data points
plt.title("Spread of Periods among LTV patients (Duplicate)")
plt.xlabel("Genes")
plt.xticks(np.arange(1,14), labels=genes)
plt.yticks(np.arange(14,37))
plt.ylabel("Period (hr)")
plt.ylim([14,36])

## Is it circadian?

In [ ]:
def CircCheck(data, i):
    # a function that classifies whether a specific gene is circadian or not
    X, Z, m = map_sub(data, i) # perform MAP estimation
    l_var = m.likelihood.variance.numpy() # extract likelihood variance
    sig_var = m.kernel.kernels[0].base_kernel.variance.numpy() # extract signal variance
    lscales = m.kernel.kernels[0].base_kernel.lengthscales.numpy() # extract lengthscales
    period = m.kernel.kernels[0].period # extract period
    if sig_var>0.1 and 20<=period<=28 and lscales>0.5:
        CircCheck = True
    else:
        CircCheck = False
    return CircCheck

#### ACU cohort

In [ ]:
CC_grid_a = np.zeros((7,13)) # create empty array to store circadian classifications for ACU cohort

for j in range(1,8): # loop over patients
    for i in range(13): # loop over genes
        CC_grid_a[j-1,i]=CircCheck(ACU_data(j),i) # classify each as circadian or not

print(CC_grid_a)

In [ ]:
CC_list_a = np.zeros(13) # create empty list to store circadian genes in for ACU cohort

for i in range(13): # loop over genes
    CC_list_a[i] = 100*((CC_grid_a[:,i] == 1).sum())/7 # calculate percentage of genes that are circadian

plt.figure(figsize=(12,8))
plt.bar(np.arange(1,14), CC_list_a)
plt.title('Percentage of Genes that are Circadian in ACU cohort')
plt.xlabel('Genes')
plt.xticks(np.arange(1,14),genes)
plt.ylabel('Percentage')

In [ ]:
CC_olist_a = np.zeros(7) # create empty list to store circadian genes in for ACU cohort

for i in range(7): # loop over patients
    CC_olist_a[i] = (CC_grid_a[i] == 1).sum() # add number of circadian genes for each patient
plt.bar(['ACU01','ACU02','ACU03','ACU04','ACU05','ACU06','ACU07'], CC_olist_a)
plt.title('Number of Circadian Genes found in each Patient')
plt.xlabel('Patients')
plt.ylabel('Number of genes')

#### LTV cohort

In [ ]:
CC_grid_l = np.zeros((6,13)) # create empty array to check whether a gene is circadian for LTV cohort
for j in range(1,7): # loop over LTV patients
    for i in range(13): # loop over genes
        CC_grid_l[j-1,i]=CircCheck(LTV_data(j),i) # classify as circadian or not

In [ ]:
CC_list_l = np.zeros(13) # create empty list to store circadian genes in for LTV cohort

for i in range(13): # loop over genes
    CC_list_l[i] = 100*((CC_grid_l[:,i] == 1).sum())/6 # calculate percentage of genes that are circadian per patient

plt.figure(figsize=(12,8))
plt.bar(np.arange(1,14), CC_list_l)
plt.title('Percentage of Genes that are Circadian in LTV Cohort')
plt.xlabel('Genes')
plt.xticks(np.arange(1,14),genes)
plt.ylabel('Percentage')

In [ ]:
CC_olist_l = np.zeros(6) # create empty list to store circadian genes for LTV cohort

for i in range(6):
    CC_olist_l[i] = (CC_grid_l[i] == 1).sum() # add circadian genes per patient
plt.bar(['LTV01','LTV02','LTV03','LTV04','LTV05','LTV06'], CC_olist_l)
plt.title('Number of Circadian Genes found in each Patient')
plt.xlabel('Patients')
plt.ylabel('Number of genes')

#### Show both cohorts

In [ ]:
CC_olist=np.concatenate((CC_olist_a,CC_olist_l)) # concatenate cohorts to plot together

plt.figure(figsize=(12,8))
plt.bar(['ACU01','ACU02','ACU03','ACU04','ACU05','ACU06','ACU07','LTV01','LTV02','LTV03','LTV04','LTV05','LTV06'], CC_olist)
plt.title('Number of Circadian Genes found in each Patient')
plt.xlabel('Patients')
plt.ylabel('Number of genes')
plt.yticks(np.arange(1,11))
plt.show()

In [ ]:
CC_list = np.zeros(13) # create list to store circadian genes
for i in range(13):
    CC_list[i] = 100*((CC_grid_a[:,i] == 1).sum()+(CC_grid_l[:,i] == 1).sum())/13 # calculate percentage of circ genes

plt.figure(figsize=(12,8))
plt.bar(np.arange(1,14), CC_list)
plt.title('Percentage of Genes that are Circadian in both cohorts')
plt.xlabel('Genes')
plt.xticks(np.arange(1,14),genes)
plt.ylabel('Percentage')